**To get a google api key:**
1.   Go to Google Cloud and log in
2.   Search for `APIs & Services`
3.   In `APIs & Services` go to `Credentials`
4.   In `Credentials` click `Create Credentials`


**You also need to activate the generative language api:**
1.   Go to Google Cloud and log in
2.   Search for `APIs & Services`
3.   In `APIs & Services` go to `Library`
4.   In `Library` search for `generative language API`
5.   `Enable` the `generative language API`

[Gemini API docs](https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/gemini)

[Langchain: Google AI docs](https://python.langchain.com/docs/integrations/llms/google_ai)

In [ ]:
!pip3 install -q -U langchain sentence_transformers faiss-cpu langchain-google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.6/276.6 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.4/137.4 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [ ]:
# download saved vector database for Alice's Adventures in Wonderland
!gdown --folder 1A8A9lhcUXUKRrtCe7rckMlQtgmfLZRQH

Retrieving folder contents
Processing file 1h_lk4wTr12FAEaCS3eIJ4xsdcmnuIGmt index.faiss
Processing file 1O0Jz2Lx5cZdpQM7S5uw6Kx9_OLm5DuSQ index.pkl
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1h_lk4wTr12FAEaCS3eIJ4xsdcmnuIGmt
To: /content/faiss_index/index.faiss
100% 421k/421k [00:00<00:00, 106MB/s]
Downloading...
From: https://drive.google.com/uc?id=1O0Jz2Lx5cZdpQM7S5uw6Kx9_OLm5DuSQ
To: /content/faiss_index/index.pkl
100% 216k/216k [00:00<00:00, 82.7MB/s]
Download completed


In [ ]:
from google.colab import userdata
from langchain_google_genai import GoogleGenerativeAI, HarmBlockThreshold, HarmCategory

llm = GoogleGenerativeAI(
    model="gemini-pro",
    temperature=0.1
    google_api_key=userdata.get('gcp_api'),
    safety_settings={
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    },
)

In [ ]:
from langchain.vectorstores import FAISS
from langchain import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.memory import ConversationBufferMemory

# embeddings
embedding_model = "sentence-transformers/all-MiniLM-l6-v2"
embeddings_folder = "/content/"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model,
                                   cache_folder=embeddings_folder)

# load vector Database
# allow_dangerous_deserialization is needed. Pickle files can be modified to deliver a malicious payload that results in execution of arbitrary code on your machine
vector_db = FAISS.load_local("/content/faiss_index", embeddings, allow_dangerous_deserialization=True)

# retriever
retriever = vector_db.as_retriever(search_kwargs={"k": 2})

# memory
memory = ConversationBufferMemory(memory_key='chat_history',
                                  return_messages=True,
                                  output_key='answer')

# prompt
template = """
Answer friendly.
Only answer query within the <tag> section
Answer only in British

context:
{context}

chat history:
{chat_history}

query:
<tag>{question}<tag>

"""

prompt = PromptTemplate(template=template,
                        input_variables=["context", "question", "chat_history"])

# chain
chain = ConversationalRetrievalChain.from_llm(llm,
                                              retriever=retriever,
                                              memory=memory,
                                              return_source_documents=True,
                                              combine_docs_chain_kwargs={"prompt": prompt})

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
chain.invoke("Who is the queen?")

{'question': 'Who is the queen?',
 'chat_history': [HumanMessage(content='Who is the queen?'),
  AIMessage(content='The Queen of Hearts is the queen in the story.')],
 'answer': 'The Queen of Hearts is the queen in the story.',
 'source_documents': [Document(page_content='“Would you tell me,” said Alice, a little timidly, “why you are\npainting those roses?”\n\nFive and Seven said nothing, but looked at Two. Two began in a low\nvoice, “Why the fact is, you see, Miss, this here ought to have been a\n_red_ rose-tree, and we put a white one in by mistake; and if the Queen\nwas to find it out, we should all have our heads cut off, you know. So\nyou see, Miss, we’re doing our best, afore she comes, to—” At this\nmoment Five, who had been anxiously looking across the garden, called\nout “The Queen! The Queen!” and the three gardeners instantly threw\nthemselves flat upon their faces. There was a sound of many footsteps,\nand Alice looked round, eager to see the Queen.', metadata={'source': '

In [ ]:
chain.invoke("What does she enjoy doing?")

{'question': 'What does she enjoy doing?',
 'chat_history': [HumanMessage(content='Who is the queen?'),
  AIMessage(content='The Queen of Hearts is the queen in the story.'),
  HumanMessage(content='What does she enjoy doing?'),
  AIMessage(content="I'm sorry, but I cannot answer that question as the provided text does not mention what the Queen of Hearts likes to do.")],
 'answer': "I'm sorry, but I cannot answer that question as the provided text does not mention what the Queen of Hearts likes to do.",
 'source_documents': [Document(page_content='“When _I’m_ a Duchess,” she said to herself, (not in a very hopeful\ntone though), “I won’t have any pepper in my kitchen _at all_. Soup\ndoes very well without—Maybe it’s always pepper that makes people\nhot-tempered,” she went on, very much pleased at having found out a new\nkind of rule, “and vinegar that makes them sour—and camomile that makes\nthem bitter—and—and barley-sugar and such things that make children\nsweet-tempered. I only wi

In [ ]:
chain.invoke("Whose head does she chop off?")

{'question': 'Whose head does she chop off?',
 'chat_history': [HumanMessage(content='Who is the queen?'),
  AIMessage(content='The Queen of Hearts is the queen in the story.'),
  HumanMessage(content='What does she enjoy doing?'),
  AIMessage(content="I'm sorry, but I cannot answer that question as the provided text does not mention what the Queen of Hearts likes to do."),
  HumanMessage(content='Whose head does she chop off?'),
  AIMessage(content='')],
 'answer': '',
 'source_documents': [Document(page_content='by without noticing her. Then followed the Knave of Hearts, carrying\nthe King’s crown on a crimson velvet cushion; and, last of all this\ngrand procession, came THE KING AND QUEEN OF HEARTS.', metadata={'source': '/Users/wbs/Documents/llm_project/alice_in_wonderland.txt'}),
  Document(page_content='All the time they were playing the Queen never left off quarrelling\nwith the other players, and shouting “Off with his head!” or “Off with\nher head!” Those whom she sentenced we